## Data Preprocessing StereoSet

In [ ]:
import pandas as pd

In [ ]:
#Loading the data:
stereo = pd.read_json('/home/hilles/stereoset/data/dev.json')

In [ ]:
stereo.info()

In [ ]:
stereo.head()

In [ ]:
print(stereo['data'][0])

In [ ]:
# Flatten the 'data' column
flat_data = pd.json_normalize(stereo['data'][0])
flat_data.head()

In [ ]:
stereo_2 = pd.read_parquet("hf://datasets/McGill-NLP/stereoset/intersentence/validation-00000-of-00001.parquet")

In [ ]:
stereo_2.info()

In [ ]:
stereo_2.head()

In [ ]:
#chatgpt
from datasets import load_dataset

# Load intersentence training split
#intersentence_train = load_dataset("McGill-NLP/stereoset", "intersentence", split="train")
#this gave an error, since huggingface doesnot include that split. Since it is for training, I am just ignoring that part

# Load intrasentence validation split
intrasentence_validation = load_dataset("McGill-NLP/stereoset", "intrasentence", split="validation")

# Similarly, load test splits for intersentence and intrasentence
#intersentence_test = load_dataset("McGill-NLP/stereoset", "intersentence", split="test")
#intrasentence_test = load_dataset("McGill-NLP/stereoset", "intrasentence", split="test")


intersentence_validation = load_dataset("McGill-NLP/stereoset", "intersentence", split="validation")

intrasentence_validation = load_dataset("McGill-NLP/stereoset", "intrasentence", split="validation")


In [ ]:
inter = pd.read_csv('/home/hilles/intersentence_validation.csv')
intra = pd.read_csv('/home/hilles/intrasentence_validation.csv') 

In [ ]:
stereoset = pd.concat([inter, intra], ignore_index=True)
stereoset.head()

In [ ]:
stereoset.info()

In [ ]:
print(stereoset['sentences'].iloc[0])

In [ ]:
print(type(stereoset['sentences'].iloc[0]))

In [ ]:
import ast

def parse_sentences(row):
    data = ast.literal_eval(row)
    sentences = data['sentence'].tolist()
    labels = data['labels']
    gold_labels = data['gold_label'].tolist()
    return pd.DataFrame({
        "sentence": sentences,
        "label": labels,
        "gold_label": gold_labels
    })

# Apply parsing to extract components
all_sentences = []
for _, row in stereoset.iterrows():
    all_sentences.append(parse_sentences(row['sentences']))

# Combine all sentences into one DataFrame
flat_sentences = pd.concat(all_sentences, ignore_index=True)
print(flat_sentences.head())

In [ ]:
import ast

# Define a parsing function
def parse_sentences(row):
    try:
        # Use ast.literal_eval to convert the string into a Python dictionary
        data = ast.literal_eval(row) 
        sentences = data['sentence']
        labels = data['labels']
        gold_labels = data['gold_label']
        
        # Create a DataFrame from the parsed data
        return pd.DataFrame({
            "sentence": sentences,
            "label": labels,
            "gold_label": gold_labels
        })
    except Exception as e:
        print(f"Error parsing row: {row}, Error: {e}")
        return pd.DataFrame()

# Test with a single example
example = stereoset['sentences'].iloc[0]
parsed_example = parse_sentences(example)
print(parsed_example)

In [ ]:
print(stereoset['target'].iloc[0])

#tried out all the other columns, and apparently the other ones are alright

In [ ]:
stereoset['sentences'].iloc[0]

In [ ]:
import pandas as pd
import ast

def expand_sentences(row):
    # Parse the sentences dictionary
    data = ast.literal_eval(row)
    sentences = data['sentence']
    ids = data['id']
    labels = data['gold_label']
    
    # Create a DataFrame for expanded sentences
    df = pd.DataFrame({
        'sentence': sentences,
        'sentence_id': ids,
        'gold_label': labels,
        'context': row['context'],
        'bias_type': row['bias_type'],
        'target': row['target']
    })
    return df

# Apply the expansion function to each row in the StereoSet DataFrame
expanded_sentences = pd.concat(
    stereoset.apply(lambda row: expand_sentences(row['sentences']), axis=1),
    ignore_index=True
)

# The expanded_sentences DataFrame now has one sentence per row


In [ ]:
#new try:

def expand_sentences(row):
    try:
        data = ast.literal_eval(row)  # Parse the sentence dictionary string
        sentences = data['sentence']
        ids = data['id']
        labels = data['gold_label']
        
        # Create a DataFrame for the expanded sentences
        df = pd.DataFrame({
            'sentence': sentences,
            'sentence_id': ids,
            'gold_label': labels,
            'context': row['context'],
            'bias_type': row['bias_type'],
            'target': row['target']
        })
        return df
    except Exception as e:
        print(f"Error parsing row: {row}. Error: {e}")
        return pd.DataFrame()  # Return an empty DataFrame for problematic rows

# Apply the expansion function
expanded_sentences = pd.concat(
    stereoset.apply(lambda row: expand_sentences(row['sentences']), axis=1),
    ignore_index=True
)


In [ ]:
stereoset.head()

In [ ]:
stereoset.info()

In [ ]:
print(stereoset['bias_type'].value_counts())